Tutorial Source:

https://www.kaggle.com/code/ludovicocuoghi/twitter-sentiment-analysis-with-bert-roberta

In [ ]:
#general purpose packages
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#data processing
import re, string
%pip install emoji
import emoji
import nltk

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split


#Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

#transformers
%pip install transformers 
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

#keras
import tensorflow as tf
from tensorflow import keras


#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

#set seed for reproducibility
seed=42

#set style for plots
sns.set_style("whitegrid")
sns.despine()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<Figure size 432x288 with 0 Axes>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test_preprocessing.csv to test_preprocessing.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving valid_preprocessing.csv to valid_preprocessing.csv


In [ ]:
df = pd.read_csv('train_preprocessing.csv',encoding='ISO-8859-1')
df_test = pd.read_csv('test_preprocessing.csv')
df_valid = pd.read_csv('valid_preprocessing.csv')

In [ ]:
df.head()

,id,date,flag,user,text,target
0,1982498874,Sun May 31 11:19:51 PDT 2009,NO_QUERY,Sun_Immaculate,miss cut grass cut grass burn smell smell burn...,0.0
1,2052542788,Sat Jun 06 01:05:28 PDT 2009,NO_QUERY,cookiemonster82,chang vote wont bat today co rain cricketsweep,1.0
2,2072011839,Sun Jun 07 20:15:00 PDT 2009,NO_QUERY,davidallentv,ty link boost even tho didnt copi whole link t...,1.0
3,2266222859,Sun Jun 21 08:07:29 PDT 2009,NO_QUERY,babyyg,whatt feeel,0.0
4,1973430757,Sat May 30 11:07:59 PDT 2009,NO_QUERY,aoitshirts,circl triangl,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960000 entries, 0 to 959999
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      960000 non-null  int64  
 1   date    960000 non-null  object 
 2   flag    960000 non-null  object 
 3   user    960000 non-null  object 
 4   text    955899 non-null  object 
 5   target  960000 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 43.9+ MB


In [ ]:
df.drop_duplicates(subset='text',inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893249 entries, 0 to 959999
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      893249 non-null  int64  
 1   date    893249 non-null  object 
 2   flag    893249 non-null  object 
 3   user    893249 non-null  object 
 4   text    893248 non-null  object 
 5   target  893249 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 47.7+ MB


In [ ]:
df = df[['text','target']]

In [ ]:
df_test = df_test[['text','target']]

In [ ]:
df_valid = df_valid[['text','target']]

In [ ]:
df['text'][1:8].values

array(['chang vote wont bat today co rain cricketsweep',
       'ty link boost even tho didnt copi whole link ty nonetheless',
       'whatt feeel', 'circl triangl',
       'feeeeel siiiick loook terribl ahhh fuck', 'huh lose',
       'glad weekend nearli here hope weather forecast right co look wall wall sun'],
      dtype=object)

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
token_lens = []

df = df.dropna()
texts = df["text"].tolist()
texts_encodings = tokenizer(texts, truncation=True, padding=True)

for txt in df['text'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    
max_len=np.max(token_lens)

In [ ]:
print(f"MAX TOKENIZED SENTENCE LENGTH: {max_len}")

MAX TOKENIZED SENTENCE LENGTH: 66


In [ ]:
token_lens = []

for i,txt in enumerate(df['text'].values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    if len(tokens)>80:
        print(f"INDEX: {i}, TEXT: {txt}")  

In [ ]:
df['token_lens'] = token_lens

In [ ]:
df = df.sort_values(by='token_lens', ascending=False)
df.head(20)

,text,target,token_lens
904026,th tweet tweet tweet tweet tweet tweet tweet t...,1.0,66
553132,good night twitter shit lol im fuuuucckedd hjd...,1.0,62
39345,stupid crazi fruiti swaag lmao wtf welll ayyyy...,1.0,61
299136,cell phone screen dead sooooooooooo text dont ...,0.0,60
879992,itali today tweet till saturday th xxxxxxxxxxx...,1.0,59
25633,f youuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuu...,0.0,59
522165,yayyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy...,1.0,57
830831,repli meeee love yooou xxxxxxxxxxxxxxxxxxxxxxx...,1.0,56
842413,franklin happi birthday happi birthday happi b...,1.0,56
436317,chaof anh chaof cac chij cac mowj cac thimsdih...,1.0,55


In [ ]:
df = df.iloc[12:]
df.head()

,text,target,token_lens
253407,hehe iy ak udh dwnload emg tp kan g afdol klo ...,0.0,54
514439,x sorrrrryyi laptop cut night night xxxxxxxxxx...,0.0,53
836844,maaf asyeeegw ga liat sm lwgw ga dpt itb syehu...,0.0,52
773831,want fukin phone aghhhhhhhhhhhhhhhhhhhhhhhhhhh...,0.0,52
913249,bukann dia repli ke akuu kmu liat dh sblm yg l...,1.0,52


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
##skipped deep cleaning
df['target'].value_counts()

0.0    449494
1.0    443742
Name: target, dtype: int64

In [ ]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['text']).reshape(-1, 1), np.array(df['target']).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['text', 'target']);

In [ ]:
train_os['target'].value_counts()

1.0    449494
0.0    449494
Name: target, dtype: int64

In [ ]:
## BERT Sentiment Analysis

MAX_LEN=128



In [ ]:
X_train = df_test['text'].values
y_train = df_test['target'].values
X_test = df_test['text'].values
y_test = df_test['target'].values
X_valid = df_valid['text'].values
y_valid = df_valid['target'].values

In [ ]:
def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [ ]:
df_test = df_test.dropna()
X_test = df_test["text"].tolist()
texts_encodings = tokenizer(X_test, truncation=True, padding=True)

df = df.dropna()
X_train = df_test["text"].tolist()
texts_encodings = tokenizer(X_train, truncation=True, padding=True)

df_valid = df_valid.dropna()
X_valid = df_valid["text"].tolist()
texts_encodings = tokenizer(X_valid, truncation=True, padding=True)

test_input_ids, test_attention_masks = tokenize(X_test, MAX_LEN)
train_input_ids, train_attention_masks = tokenize(X_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize(X_valid, MAX_LEN)


In [ ]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def create_model(bert_model, max_len=MAX_LEN):
    
    ##params###
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.CategoricalCrossentropy()
    accuracy = tf.keras.metrics.CategoricalAccuracy()


    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    embeddings = bert_model([input_ids,attention_masks])[1]
    
    output = tf.keras.layers.Dense(3, activation="softmax")(embeddings)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)
    
    model.compile(opt, loss=loss, metrics=accuracy)
    
    
    return model

In [ ]:
model = create_model(bert_model, MAX_LEN)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  109482240   ['input_3[0][0]',                
                                thPoolingAndCrossAt               'input_4[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         

In [ ]:
history_bert = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=4, batch_size=32)

ValueError: ignored

In [ ]:
## BERT results
result_bert = model.predict([test_input_ids,test_attention_masks])


  19/9958 [..............................] - ETA: 37:48:28

KeyboardInterrupt: ignored

In [ ]:
y_pred_bert =  np.zeros_like(result_bert)
y_pred_bert[np.arange(len(y_pred_bert)), result_bert.argmax(1)] = 1